In [29]:
import tensorflow as tf

from tensorflow.keras.datasets import mnist, cifar10, cifar100

from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation

from sklearn.model_selection import train_test_split

import numpy as np
import random

import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

In [2]:
#load the data
(x, y), (x_test, y_test) = mnist.load_data()

labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
img_rows, img_cols, channels = 28, 28, 1
num_classes = 10

#process the data
x, x_test = x / 255.0, x_test / 255.0

x = x.reshape((-1, img_rows, img_cols, channels))
x_test = x_test.reshape((-1, img_rows, img_cols, channels))

y = tf.keras.utils.to_categorical(y, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

#split training set, test set, attack set
x_train, x_attack, y_train, y_attack = train_test_split(x, y, test_size=0.1)

print("Data shapes:", x_test.shape, y_test.shape, x_train.shape, y_train.shape, x_attack.shape, y_attack.shape)

Data shapes: (10000, 28, 28, 1) (10000, 10) (54000, 28, 28, 1) (54000, 10) (6000, 28, 28, 1) (6000, 10)


In [3]:
#training model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

victim_model = create_model()
victim_model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data = (x_test, y_test))
victim_model.save('model/victim_model')


Epoch 1/10
1688/1688 [==============================] - 5s 2ms/step - loss: 0.0304 - accuracy: 0.7838 - val_loss: 0.0094 - val_accuracy: 0.9385
Epoch 2/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0089 - accuracy: 0.9412 - val_loss: 0.0062 - val_accuracy: 0.9602
Epoch 3/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0066 - accuracy: 0.9568 - val_loss: 0.0059 - val_accuracy: 0.9611
Epoch 4/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0055 - accuracy: 0.9643 - val_loss: 0.0054 - val_accuracy: 0.9623
Epoch 5/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0048 - accuracy: 0.9679 - val_loss: 0.0048 - val_accuracy: 0.9688
Epoch 6/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0045 - accuracy: 0.9712 - val_loss: 0.0045 - val_accuracy: 0.9695
Epoch 7/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0039 - accuracy: 0.9749 - val_loss: 0.0055 - val_accuracy:

In [4]:
#testing model
print("Base accuracy on regular images:", victim_model.evaluate(x=x_test, y=y_test, verbose=0))

Base accuracy on regular images: [0.004416625946760178, 0.9718999862670898]


In [5]:
#prediction API
def get_prediction(image):
    return victim_model.predict(image).argmax()

In [45]:
#Collect initial data
x_attack_init_ind = random.sample(range(0, x_attack.shape[0]), 100)
x_attack_init = x_attack[x_attack_init_ind]

#Label initial data
y_attack_init = []
for image in x_attack_init:
    y_attack_init.append(get_prediction(image.reshape(1, img_rows, img_cols, channels)))

y_attack_init = tf.one_hot(y_attack_init, 10)

In [46]:
#Create attacker model architecture
def create_attack_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

attack_model = create_attack_model()
attack_model.fit(x_attack_init, y_attack_init, batch_size=32, epochs=10, validation_data = (x_test, y_test))
print("Attack model accuracy: %f" % (attack_model.evaluate(x_test, y_test)[1]))

Epoch 1/10
4/4 [==============================] - 1s 151ms/step - loss: 0.0898 - accuracy: 0.1188 - val_loss: 0.0896 - val_accuracy: 0.1197
Epoch 2/10
4/4 [==============================] - 0s 103ms/step - loss: 0.0891 - accuracy: 0.1668 - val_loss: 0.0894 - val_accuracy: 0.1852
Epoch 3/10
4/4 [==============================] - 0s 104ms/step - loss: 0.0882 - accuracy: 0.3073 - val_loss: 0.0891 - val_accuracy: 0.1972
Epoch 4/10
4/4 [==============================] - 0s 105ms/step - loss: 0.0877 - accuracy: 0.2618 - val_loss: 0.0888 - val_accuracy: 0.1880
Epoch 5/10
4/4 [==============================] - 0s 107ms/step - loss: 0.0866 - accuracy: 0.2522 - val_loss: 0.0882 - val_accuracy: 0.1678
Epoch 6/10
4/4 [==============================] - 0s 117ms/step - loss: 0.0855 - accuracy: 0.2316 - val_loss: 0.0869 - val_accuracy: 0.1860
Epoch 7/10
4/4 [==============================] - 0s 104ms/step - loss: 0.0842 - accuracy: 0.2373 - val_loss: 0.0853 - val_accuracy: 0.2192
Epoch 8/10
4/4 [====

In [47]:
#creating adversarial example
def adversarial_pattern(image, label, model):
    image = tf.cast(image, tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.MSE(label, prediction)
    
    gradient = tape.gradient(loss, image)
    
    signed_grad = tf.sign(gradient)
    
    return signed_grad

In [48]:
#Create synthetic example
def train_attack_model(iteration, eps):
    counter = 1
    x_attack_set = x_attack_init
    y_attack_set = y_attack_init
    
    while True:
        print("Iteration: %d" % (counter))

        for i in range(x_attack_set.shape[0]):
            image = x_attack_set[i]
            image_label = y_attack_set[i]
            perturbations = adversarial_pattern(image.reshape((1, img_rows, img_cols, channels)), image_label, attack_model).numpy()
            adversarial = image + perturbations * eps
            x_attack_set = np.append(x_attack_set, adversarial, axis = 0)

        #Label new dataset
        y_attack_set = []
        for image in x_attack_set:
            y_attack_set.append(get_prediction(image.reshape(1, img_rows, img_cols, channels)))

        y_attack_set = tf.one_hot(y_attack_set, 10)

        #train the attack model with new dataset
        attack_model.fit(x_attack_set, y_attack_set, batch_size=32, epochs=10, validation_data = (x_test, y_test))

        #evluate the attack model
        print("Dataset size: %d" % (x_attack_set.shape[0]))
        print("Attack model accuracy: %f" % (attack_model.evaluate(x_test, y_test)[1]))

        if counter == iteration:
            break

        counter += 1

train_attack_model(3, 64/255.0)

Iteration: 1
Epoch 1/10
7/7 [==============================] - 0s 65ms/step - loss: 0.0811 - accuracy: 0.3000 - val_loss: 0.0748 - val_accuracy: 0.4164
Epoch 2/10
7/7 [==============================] - 0s 55ms/step - loss: 0.0773 - accuracy: 0.3650 - val_loss: 0.0722 - val_accuracy: 0.4659
Epoch 3/10
7/7 [==============================] - 0s 55ms/step - loss: 0.0737 - accuracy: 0.4450 - val_loss: 0.0689 - val_accuracy: 0.4985
Epoch 4/10
7/7 [==============================] - 0s 51ms/step - loss: 0.0709 - accuracy: 0.4850 - val_loss: 0.0635 - val_accuracy: 0.5922
Epoch 5/10
7/7 [==============================] - 0s 54ms/step - loss: 0.0665 - accuracy: 0.5200 - val_loss: 0.0602 - val_accuracy: 0.5877
Epoch 6/10
7/7 [==============================] - 0s 53ms/step - loss: 0.0625 - accuracy: 0.5050 - val_loss: 0.0552 - val_accuracy: 0.6288
Epoch 7/10
7/7 [==============================] - 0s 55ms/step - loss: 0.0595 - accuracy: 0.5550 - val_loss: 0.0534 - val_accuracy: 0.6182
Epoch 8/10
7/7